In [15]:
import numpy as np
from scipy.stats import rayleigh
import sys
import os
import pandas as pd
import numpy as np
from matplotlib import animation
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import matplotlib

Смотрим на то, где находится корень проекта.

In [16]:
os.getcwd()

'/home/michael/SBDPP_sim/examples'

Указываем корень проекта.

In [17]:
sys.path.append("/home/michael/SBDPP_sim") # ваш путь к папке с проектом
import simulation

In [ ]:
L = 50.0
M = 4

r_values = np.arange(0, 5.01, 0.01)

# 2D standard normal distribution radial density
# The radial distribution for a 2D normal is proportional to Rayleigh distribution with scale 1 (standard normal case)
def density(r):
    return (1 / (2 * np.pi)) * np.exp(-0.5 * r**2)

radial_density_values = density(r_values)
q_values = np.arange(0, 1.0, 0.001)
inverse_radial_values = rayleigh.ppf(q_values, scale=1.0)
g2 = simulation.PyGrid2(
    M=M,
    areaLen=[L, L],
    cellCount=[L, L],
    isPeriodic=True,
    birthRates=[1]*M,
    deathRates=[0]*M,
    ddMatrix=[0.1, 0.3, 0.3, 0.3,
              0.3, 0.1, 0.3, 0.3,
              0.3, 0.3, 0.1, 0.3,
              0.3, 0.3, 0.3, 0.1,],
    birthX=[q_values.tolist()]*M,
    birthY=[inverse_radial_values.tolist()]*M,
    deathX_=[ [ r_values.tolist()]*M]*M,
    deathY_=[ [ radial_density_values.tolist() ]*M ]*M,
    cutoffs=[5.0]*M*M,
    seed=42,
    rtimeLimit=7200.0
)


# Parameters
N = 1000  # Number of [x, y] pairs per group

# Generate the list
coordinates = []
for _ in range(M):
    group = [[np.random.uniform(0, L), np.random.uniform(0, L)] for _ in range(N)]
    coordinates.append(group)

g2.placePopulation(coordinates)

print("Birth rate:", g2.total_birth_rate)
print("Death rate:", g2.total_death_rate)

Birth rate: 4000.0
Death rate: 1604.9713441548117


In [19]:
data = list()

for t in range(30):
    for c in range(g2.get_num_cells()):
        for n in range(M):
            coords = g2.get_cell_coords(c,n)
            for i in range(len(coords)):
                x,y = coords[i]
                data.append([t,x,y,n])
    g2.run_for(1)
    print(t, g2.total_population)

0 6300
1 8499
2 9641
3 10602
4 11231
5 11677
6 12166
7 12823
8 13713
9 14289
10 15297
11 16340
12 17128
13 17794
14 18567
15 18940
16 19228
17 19640
18 20094
19 20501
20 20592
21 20630
22 20724
23 20800
24 20836
25 20941
26 20985
27 20852
28 21042
29 21333


In [20]:
df = pd.DataFrame(columns = ['time', 'x', 'y', 's'], data = data)
display(df.set_index('time'))

,x,y,s
time,,,
0,0.201075,0.360439,3
0,1.666550,0.515205,1
0,1.865654,0.631132,2
0,2.658599,0.255019,2
0,2.127452,0.859779,2
...,...,...,...
29,49.246210,49.075874,2
29,49.319927,49.688617,2
29,49.423249,49.478240,2


In [21]:
movie_writer = animation.FFMpegWriter(fps=2, codec='h264', extra_args=['-vcodec', 'libx264'])

In [22]:

matplotlib.rcParams['animation.ffmpeg_path'] = '/usr/bin/ffmpeg' # путь к ffmpeg

df = df.sort_values('time')

output_file = "points_animation.mp4"

fig, ax = plt.subplots(figsize=(8, 8))

ax.set_xlim(0, L)
ax.set_ylim(0, L)
ax.set_xlabel('x')
ax.set_ylabel('y')
sc = ax.scatter([], [], c=[], cmap='plasma', s=20, alpha=0.6)

def update(frame):
    current_df = df[df['time'] == frame]
    sc.set_offsets(current_df[['x', 'y']].values)
    sc.set_array(current_df['s'].values)
    ax.set_title(f"Time: {frame}")
    return sc,

ani = FuncAnimation(fig, update, frames=df['time'].nunique(), blit=True)

ani.save(output_file, writer=movie_writer)
print(f"Animation saved as {output_file}")
plt.close(fig)

Animation saved as points_animation.mp4
